https://adventofcode.com/2022/day/11

In [1]:
f = open("input.txt", "r")

data = read(f, String)

close(f)

In [2]:
mutable struct Monkey
	name
	items
	operation
	test_factor
	getNextMonkeyName
	inspect_count
end

function inspect(item, monkey::Monkey)
	global old = item #set the global scoped old
	eval(monkey.operation) # evaluates the string as global scoped Julia code
	monkey.inspect_count += 1 # increments monkey's inspect count
	return new
end

function throw(monkey::Monkey, monkey_map)
	item = popfirst!(monkey.items)
	item = inspect(item, monkey)
	item = item ÷ 3 # floor divide by 3

	next_monkey_name = monkey.getNextMonkeyName(item) # calculate which monkey to throw to
	next_monkey = monkey_map[next_monkey_name]	
	
	push!(next_monkey.items, item)
end

function generateMonkey(monkey)::Monkey
	name = match(r"(?<=Monkey )\d+",monkey).match

	items = match(r"(?<=Starting items: ).*",monkey).match
	items = split(items,", ")
	items = map(item -> parse(Int,item), items)

	operation = match(r"(?<=Operation: ).*",monkey).match
	operation = Meta.parse(operation)

	test_factor = match(r"(?<=Test: divisible by )\d+",monkey).match
	test_factor = parse(Int,test_factor)

	true_monkey = match(r"(?<=If true: throw to monkey )\d+",monkey).match
	false_monkey = match(r"(?<=If false: throw to monkey )\d+",monkey).match

	# calculate which monkey to throw to
	getNextMonkeyName(item) = item % test_factor === 0 ? true_monkey : false_monkey

	return Monkey(name, items, operation, test_factor, getNextMonkeyName,0)
end

# creates an array and a map of the monkeys
function generateMonkeys(data)
	all_monkey_data = split(data,r"\n\n")
	monkey_map = Dict()
	monkeys = []
	for monkey_data in all_monkey_data
		# create a monkey object from the monkey_data string
		monkey = generateMonkey(monkey_data)
		monkey_map[monkey.name] = monkey
		push!(monkeys, monkey)
	end

	return monkeys, monkey_map
end

# calculates and sets the state of the monkeys in 1 round
function simulateRound(monkeys, monkey_map)
	for monkey in monkeys
		for i in 1:length(monkey.items)
			throw(monkey, monkey_map)
		end
	end
end

# calculates the monkey business
function getMonkeyBusiness(test)
	monkeys, monkey_map = generateMonkeys(test)
	for i = 1:20
		simulateRound(monkeys, monkey_map)
	end

	monkey_business=map(monkey -> monkey.inspect_count,monkeys)
	sort!(monkey_business, rev=true)

	return monkey_business[1]*monkey_business[2]
end


old = 0 # create a globally scoped old variable
new = 0 # create a globally scoped new variable

getMonkeyBusiness(data)

121450

In [3]:
function throw2(monkey::Monkey, monkey_map, reduce_mod)
	item = popfirst!(monkey.items)
	item = inspect(item, monkey)
	item %= reduce_mod # reduces the item lower than the product of the all the monkey test_factors

	next_monkey_name = monkey.getNextMonkeyName(item)
	next_monkey = monkey_map[next_monkey_name]	
	
	push!(next_monkey.items, item)
end

function simulateRound2(monkeys, monkey_map, reduce_mod)
	for monkey in monkeys
		for i in 1:length(monkey.items)
			throw2(monkey, monkey_map,reduce_mod)
		end
	end
end

#parses the monkey using BigInt
function generateMonkey2(monkey)::Monkey
	name = match(r"(?<=Monkey )\d+",monkey).match

	items = match(r"(?<=Starting items: ).*",monkey).match
	items = split(items,", ")
	items = map(item -> parse(BigInt,item), items)

	operation = match(r"(?<=Operation: ).*",monkey).match
	operation = Meta.parse(operation)

	test_factor = match(r"(?<=Test: divisible by )\d+",monkey).match
	test_factor = parse(BigInt,test_factor)

	true_monkey = match(r"(?<=If true: throw to monkey )\d+",monkey).match
	false_monkey = match(r"(?<=If false: throw to monkey )\d+",monkey).match

	getNextMonkeyName(item) = item % test_factor == 0 ? true_monkey : false_monkey

	return Monkey(name, items, operation, test_factor, getNextMonkeyName,0)
end

function generateMonkeys2(data)
	all_monkey_data = split(data,r"\n\n")
	monkey_map = Dict()
	monkeys = []
	for monkey_data in all_monkey_data
		monkey = generateMonkey2(monkey_data)
		monkey_map[monkey.name] = monkey
		push!(monkeys, monkey)
	end

	return monkeys, monkey_map
end

function getMonkeyBusiness2(test)
	monkeys, monkey_map = generateMonkeys(test)
	# calculates the product of the all the monkey test_factors
	reduce_mod = reduce((mod,monkey) -> mod*monkey.test_factor, monkeys, init=1)

	for i = 1:10000
		simulateRound2(monkeys, monkey_map,reduce_mod)
	end

	monkey_business=map(monkey -> monkey.inspect_count,monkeys)
	sort!(monkey_business, rev=true)

	return monkey_business[1]*monkey_business[2]
end

# sets the global variables to type BigInt
old = BigInt(0)
new = BigInt(0)

getMonkeyBusiness2(data)

28244037010